Analysis of the impact of samplesize for exploratory landscape analysis features

In [ ]:
import cocoex
from pflacco.classical_ela_features import *
from pflacco.sampling import create_initial_sample

_features = []
# Get all 24 single-objective noiseless BBOB function in dimension 2 and 3 for the first five instances.
suite = cocoex.Suite("bbob", f"instances:1-5", f"function_indices:1-24 dimensions:2,3,5")
for problem in suite:
    dim = problem.dimension
    fid = problem.id_function
    iid = problem.id_instance
    #for sample_coefficient in [1, 50]:
    for n in [10,20,50]:
        #for i in range(100):
            

        # Create sample
        #X = create_initial_sample(dim, lower_bound = -5, upper_bound = 5, sample_coefficient=sample_coefficient, sample_type='lhs')
        X = create_initial_sample(dim, lower_bound = -5, upper_bound = 5, n=n, sample_type='lhs')
        y = X.apply(lambda x: problem(x), axis = 1)

        # Calculate ELA features
        ela_meta = calculate_ela_meta(X, y)
        ela_distr = calculate_ela_distribution(X, y)
        fast_k = max(math.ceil(0.05 * X.shape[0]),2)
        nbc = calculate_nbc(X, y, fast_k=fast_k)
        disp = calculate_dispersion(X, y)
        ic = calculate_information_content(X, y, seed = 100)
        pca = calculate_pca(X, y)
        #try:
        #    level = calculate_ela_level(X, y)
        #except Exception:
        #    level = {}

        # Store results in pandas dataframe ### **{'sample_coefficient': sample_coefficient}
        data = pd.DataFrame({**ic, **ela_meta, **ela_distr, **nbc, **disp, **pca, **{'fid': fid}, **{'dim': dim}, **{'iid': iid}, **{'n': n}}, index = [0])
        _features.append(data)
        

features_test = pd.concat(_features).reset_index(drop = True)

In [ ]:
features.to_parquet('df.lhs.gzip', compression='gzip')

In [ ]:
import cocoex
from pflacco.classical_ela_features import *
from pflacco.sampling import create_initial_sample
features = pd.read_parquet('df.lhs.gzip')

In [ ]:
from scipy.optimize import minimize
import cma

import cocoex
from pflacco.classical_ela_features import *
from pflacco.sampling import create_initial_sample

optimizers = ['BFGS', 'L-BFGS-B', 'SLSQP', cma.CMAEvolutionStrategy]
func_evals = []
# Get all 24 single-objective noiseless BBOB function in dimension 2 and 3 for the first five instances.
suite = cocoex.Suite("bbob", f"instances:1-5", f"function_indices:1-24 dimensions:2,3,5,10")
for problem in suite:
    dim = problem.dimension
    fid = problem.id_function
    iid = problem.id_instance
    
    for optim in optimizers:
        for i in range(5):
            x0 = problem.initial_solution_proposal(i)
            
            if optim is str:
                sol = minimize(problem, x0, method=optim)
                func_evals.append(pd.DataFrame({'fid': fid, 'dim': dim, 'iid': iid, 'optim': optim, **dict(enumerate(x0)),'fun': sol.fun, 'nfev': sol.nfev, 'status': sol.status}, index = [0]))
            elif optim is cma.CMAEvolutionStrategy:
                x, es = cma.fmin2(problem, x0, 0.5)
                sol = es.result._asdict()
                func_evals.append(pd.DataFrame({'fid': fid, 'dim': dim, 'iid': iid, 'optim': 'CMA-ES', **dict(enumerate(x0)),'fun': sol['fbest'], 'nfev': sol['evaluations']}, index = [0]))
func_evals = pd.concat(features).reset_index(drop = True)

In [ ]:
func_evals.columns = func_evals.columns.astype(str)
func_evals.to_parquet('func_evals.gzip', compression='gzip')

In [ ]:
func_evals = pd.read_parquet('func_evals.gzip')

Read in the benchmark and find the optimal minimizer within an error margin

In [ ]:
benchmark = pd.read_csv('imputed_relERT_costs_50d.csv').drop(['repetition'], axis=1).set_index(['dim', 'fid'])
deviation = 0.01
mins = benchmark.min(axis=1) * (1+deviation)
#benchmark = benchmark.le(mins, axis='index').melt(value_vars=benchmark.columns, var_name='optimizer', ignore_index=False)
benchmark = benchmark[benchmark.le(mins, axis='index')].melt(value_vars=benchmark.columns, var_name='optimizer', value_name='niter', ignore_index=False).dropna().sort_index()
cat_mapping = benchmark.optimizer.astype('category').cat.categories.to_list()
#cat_mapping = dict(zip(range(len(cat_mapping)), cat_mapping))
benchmark.optimizer = benchmark.optimizer.astype('category').cat.codes


In [ ]:
from sklearn.ensemble import HistGradientBoostingClassifier, GradientBoostingClassifier


infeatures_fix = ['ic.h_max', 'ic.eps_s', 'ic.eps_max', 'ic.eps_ratio', 'ic.m0',
       'ela_meta.lin_simple.adj_r2', 'ela_meta.lin_simple.intercept',
       'ela_meta.lin_simple.coef.min', 'ela_meta.lin_simple.coef.max',
       'ela_meta.lin_simple.coef.max_by_min',
       'ela_meta.lin_w_interact.adj_r2', 'ela_meta.quad_simple.adj_r2',
       'ela_meta.quad_simple.cond', 'ela_meta.quad_w_interact.adj_r2',
       'ela_distr.skewness', 'ela_distr.kurtosis',
       'ela_distr.number_of_peaks', 'nbc.nn_nb.sd_ratio',
       'nbc.nn_nb.mean_ratio', 'nbc.nn_nb.cor',
       'nbc.dist_ratio.coeff_var', 'nbc.nb_fitness.cor',
       'disp.ratio_mean_25', 'disp.ratio_median_25', 'disp.diff_mean_25',
       'disp.diff_median_25']

levels = features.columns[features.columns.str.contains('ela_level')].to_list()
pca = features.columns[features.columns.str.contains('pca')].to_list()

clf = GradientBoostingClassifier()
_slice = features.loc[features.dim == 2,:].dropna(axis=1)
_slice = _slice.loc[:,~_slice.columns.str.contains('costs_runtime')]
X = _slice.drop(['fid', 'dim', 'iid', 'sample_coefficient', 'n'] + levels +pca, axis=1, errors='ignore').replace([np.inf], 99999999)#.to_numpy()
X = _slice.loc[:,infeatures_fix].replace([np.inf], 99999999)
y = _slice.loc[:,'fid'].to_numpy()-1
clf = clf.fit(X,y)

##Eval
clf_test = GradientBoostingClassifier()
_slice = features_test.loc[features_test.dim == 2,:].dropna(axis=1)
_slice = _slice.loc[:,~_slice.columns.str.contains('costs_runtime')]
#levels = _slice.columns[_slice.columns.str.contains('ela_level')].to_list()
#X = _slice.drop(['fid', 'dim', 'iid', 'sample_coefficient', 'n'] + levels +pca, axis=1, errors='ignore').replace([np.inf], 99999999)#.to_numpy()
X = _slice.loc[:,infeatures_fix].replace([np.inf], 99999999)
y = _slice.loc[:,'fid'].to_numpy()-1
clf_test = clf_test.fit(X,y)



Set up RL Agent with custom feature exctractor

In [ ]:
import gym
import numpy as np
from gym import spaces
from numpy.random import default_rng
from stable_baselines3 import A2C, DQN
from stable_baselines3.common.callbacks import EvalCallback
from stable_baselines3.common.vec_env import SubprocVecEnv, VecNormalize, DummyVecEnv, VecCheckNan
from stable_baselines3.common.env_util import make_vec_env
from stable_baselines3.common.utils import set_random_seed


import torch.nn as nn
import torch.nn.functional as F
import torch
import random

from itertools import permutations
from stable_baselines3.common.torch_layers import BaseFeaturesExtractor


class ClassifierEnv(gym.Env):
    """Classifier Environment for ELA based AAS."""

    metadata = {"render_modes": ["human"], "render_fps": 30}

    def __init__(self, classifier, truth, used_cols, fid='1-24', iid='1-5', dim=2, problem='bbob'):
        super().__init__()
        # Define action and observation space
        # They must be gym.spaces objects
        # Example when using discrete actions:
        #self.problem_def = (problem, f"instances:{iid}", f"function_indices:{fid} dimensions:{dim}")
        
        self.suite = None# cocoex.Suite(problem, problem_select)
        self.classifier = classifier
        self.dim = dim
        self.truth = truth
        self.fid = fid
        self.iid = iid
        self.rng = default_rng()
        self.used_cols = used_cols
        self.random = True
        
        self.action_space = spaces.Discrete(2)
        # Example for using image as input (channel-first; channel-last also works):
        self.observation_space = spaces.Box(-np.inf, np.inf, shape=(1,len(used_cols)), dtype=np.float32)
    
    @staticmethod
    def sigmoid(x):
        return 1/(1+math.exp(-0.1*x))
    
    @staticmethod
    def softplus(x):
        return math.ln(1.0+math.exp(x))-1.0
    
    @staticmethod
    def elu(x, alpha=1.0):
        return x if x>0.0 else alpha*(math.exp(x)-1.0)
    
    def step(self, action):
        if action == 0:
            #X = create_initial_sample(self.dim, lower_bound = -5, upper_bound = 5, n=10, sample_type='random')
            cols = [f"x{i}" for i in range(self.dim)]
            X = pd.DataFrame(self.rng.random((10, self.dim))*10-5, columns=cols)
            self.X = pd.concat([self.X, X], ignore_index=True)

            observation = self.calc_ela(self.X)
            if np.isnan(observation).any():
                raise ValueError(f"Nan err with {self.X}")
            reward = -1.0
            done = False
        elif action == 1:
            #problem ready for AAS
            observation = self.calc_ela(self.X)
            #print(observation)
            predict = self.classifier.predict(observation)[0]
            fid = self.problem.id_function
            dim = self.problem.dimension
            mask = self.truth.loc[(dim,fid),'optimizer'] == predict
            
            if mask.any():
                #print(self.truth.loc[(dim,fid),:])
                niter = self.truth.loc[(dim,fid),'niter'][mask][0] #.values[0]
                reward = self.elu(niter-self.X.shape[0])
            else:
                niter = self.truth.loc[(dim,fid),'niter'].min()
                reward = -self.elu(niter-self.X.shape[0]) - 2
            
            done = True
            
            
        else:
            raise ValueError(
                f"Received invalid action={action} which is not part of the action space"
            )
            
        truncated = False
        info = {}
        
        return observation, reward, done, info
    
        
    
    def calc_ela(self, X):
        
        y = X.apply(lambda x: self.problem(x), axis = 1)
        
        ela_meta = calculate_ela_meta(X, y)
        ela_distr = calculate_ela_distribution(X, y)
        fast_k = max(math.ceil(0.05 * X.shape[0]),2)
        try:
            nbc = calculate_nbc(X, y, fast_k=fast_k)
        except IndexError as e:
            print(X)
            print(y)
            raise e
        disp = calculate_dispersion(X, y)
        #try:
        #    ic = calculate_information_content(X, y, seed = 100)
        #except KeyError as e:
        #    assert (X.index==y.index).all()
        #    print(X)
        #    print(y)
        #    raise e
        ##pca = calculate_pca(X, y)
        #try:
        #    level = calculate_ela_level(X, y)
        #except Exception:
        #    level = {}
        data = {**ic, **ela_meta, **ela_distr, **nbc, **disp, } #**pca
        del data['ela_meta.costs_runtime']
        #del data['pca.costs_runtime']
        del data['nbc.costs_runtime']
        del data['disp.costs_runtime']
        del data['ic.costs_runtime']
        del data['ela_distr.costs_runtime']
        #del data['limo.costs_runtime']
        #del data['cm_angle.costs_runtime']
        #del data['cm_conv.costs_runtime']
        #del data['cm_grad.costs_runtime']
        #del data['ela_conv.costs_runtime']
        #del data['ela_level.costs_runtime']
        #del data['ela_curv.costs_runtime']
        #del data['ela_local.costs_runtime']
        #data = {k,v for k,v in data.items() if not "costs_runtime" in k}
        
        data = {k:v for k,v in data.items() if k in self.used_cols}
        result = np.array(list(data.values()), dtype=np.float32).reshape(1, -1)
        result[np.isnan(result)] = 0#-(2**32)#-np.inf
        print(result)
        print(result.shape)
        return result#torch.nan_to_num(result, nan=-(2.0**32))
        #return {**ic, **ela_meta, **ela_distr, **nbc, **disp, **pca}#**level
    
    def next_problem(self, options=None):
        self.X = None
        if self.random:
            fid = random.choice(self.funcs)
            self.problem = self.suite.get_problem(fid)
        else:
            self.problem = next(self.problem_selector)
        self.X = create_initial_sample(self.dim, lower_bound = -5, upper_bound = 5, n=10, sample_type='lhs')

        observation = self.calc_ela(self.X)
        return observation#, info
    
    def reset(self, fid='1-24', iid='1-5', seed=0, n_samples=10, options=None):
        
        #problem_def = ('bbob', f"instances:{iid}", f"function_indices:{fid} dimensions:{dim}")
        self.suite = cocoex.Suite('bbob', f"instances:{iid}", f"function_indices:{fid} dimensions:{self.dim}")
        self.problem_selector = iter(self.suite)
        self.funcs = self.suite.ids()
        
        return self.next_problem()
    
    def render(self):
        pass

    def close(self):
        pass
    
    
class SimpelConvNet(BaseFeaturesExtractor):
    """
    :param observation_space: (gym.Space)
    :param features_dim: (int) Number of features extracted.
        This corresponds to the number of unit for the last layer.
    """
    
    def __init__(self, observation_space: spaces.Box, features_dim: int = 26):
        super().__init__(observation_space, features_dim)
        #NxCxL batchXchannelXfeatures
        n_channels = observation_space.shape[0]
        self.cnn = nn.Sequential(
            nn.Conv1d(n_channels, 16, kernel_size=4),
            nn.ReLU(),
            nn.Conv1d(16, 32, kernel_size=4),#, dilation=3
            nn.ReLU(),
            nn.Conv1d(32, 64, kernel_size=4),
            nn.ReLU(),
            nn.Flatten(),
        )
        with torch.no_grad():
            #print(observation_space.sample()[:,None].shape)
            n_flatten = self.cnn(
                torch.as_tensor(observation_space.sample()[:,None]).float()
            ).shape[1]
        print(n_flatten)
        self.linear = nn.Sequential(nn.Linear(n_flatten, features_dim), nn.ReLU())
        
    def forward(self, observations: torch.Tensor) -> torch.Tensor:
        return self.linear(self.cnn(observations))
    
class SkipNetwork(BaseFeaturesExtractor):
    """
    :param observation_space: (gym.Space)
    :param features_dim: (int) Number of features extracted.
        This corresponds to the number of unit for the last layer.
    """
    
    def __init__(self, observation_space: spaces.Box, features_dim: int = 26):
        super().__init__(observation_space, features_dim)
        #NxCxL batchXchannelXfeatures
        n_channels = observation_space.shape[0]
        self.cnn = nn.Sequential(
            nn.Conv1d(n_channels, 16, kernel_size=4),
            nn.ReLU(),
            nn.Conv1d(16, 32, kernel_size=4, dilation=3),
            nn.ReLU(),
            nn.Flatten(),
        )
        self.conv1 = nn.Conv1d(n_channels, 16, kernel_size=4)
        self.conv2 = nn.Conv1d(16, 32, kernel_size=4, dilation=3)
        self.flatten = nn.Flatten()
        with torch.no_grad():
            #print(observation_space.sample()[:,None].shape)
            n_flatten = self.flatten(self.conv2(self.conv1(
                torch.as_tensor(observation_space.sample()[:,None]).float()
            ))).shape[1] + features_dim
        print(f"feat:{features_dim}")
        self.linear = nn.Sequential(nn.Linear(n_flatten, features_dim), nn.ReLU())
        
    def forward(self, observations: torch.Tensor) -> torch.Tensor:
        #out = observations.copy()
        x = self.conv1(observations)
        x = nn.ReLU()(x)
        x = self.conv2(x)
        x = nn.ReLU()(x)
        x = self.flatten(x)
        x = torch.cat((x, self.flatten(observations)), 1)
        
        return self.linear(x)
    
from stable_baselines3.common.env_checker import check_env

def make_env(rank, seed=0):
    def _init():
        env = ClassifierEnv(clf, benchmark[~benchmark.index.duplicated(keep='first')], infeatures_fix, dim=rank+2)
        env.seed(rank)
        return env
    set_random_seed(seed)
    return _init

#env = ClassifierEnv(clf, benchmark[~benchmark.index.duplicated(keep='first')], infeatures_fix)

num_cpu = 1
#env = make_env(1)()
env = SubprocVecEnv([make_env(i) for i in range(1)])
env = VecNormalize(env, training=True, norm_obs=True, norm_reward=True)
#env = VecCheckNan(env, raise_exception=True)
obs = env.reset()
#model = A2C("MlpPolicy", env, learning_rate=0.0007, verbose=1, tensorboard_log="./dqn_AAS_tensorboard/")
#policy_kwargs = dict(net_arch=dict(pi= [16,16], vf=[16,16]))
policy_kwargs = dict(
    features_extractor_class=SimpelConvNet,
    features_extractor_kwargs=dict(features_dim=obs.shape[2])
)#dict(net_arch=[64,64,16])
#print(obs.shape)
model = A2C("MlpPolicy", env, policy_kwargs=policy_kwargs, tensorboard_log="./dqn_AAS_tensorboard/", verbose=1)

In [ ]:
    

def make_evalenv(rank, seed=0):
    def _init():
        env = ClassifierEnv(clf_test, benchmark[~benchmark.index.duplicated(keep='first')], infeatures_fix, dim=rank+2)
        env.seed(rank)
        return env
    set_random_seed(seed)
    return _init


eval_env = DummyVecEnv([make_evalenv(i, seed=3) for i in range(1)])
#eval_env = DummyVecEnv(eval_env)
eval_env = VecNormalize(eval_env, training=True, norm_obs=True, norm_reward=True)
eval_callback = EvalCallback(eval_env, log_path="./logs/", eval_freq=50,
                            deterministic=True, render=False)

model.learn(total_timesteps=10_000)#, callback=eval_callback)


In [ ]:
env.step([0])

In [ ]:
infeatures_fix[12]

In [ ]:
benchmark

In [ ]:
obs = env.reset('1-24', '1-5')
for i in range(100):
    action, _state = model.predict(obs, deterministic=True)
    obs, reward, done, info = env.step(action)
    if done:
        print(f"Done in {i}")
        break
        
        obs = env.next_problem()

In [ ]:
y

In [ ]:
%matplotlib inline
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import seaborn as sns
def func(x):
    y = np.where(x <3, np.log(1+np.exp(x)), 0)
    #
    mask = np.insert(np.diff(y),0,0)
    y[mask < 0.0] = np.nan
    return y
def func2(x):
    y = np.where(x<3, np.where(x>0, x, np.exp(x)-1), -x)
    #y = #np.where(x>3, np.where(x>0, -x, np.exp(x)-1), np.exp(x)-1)
    mask = np.insert(np.diff(y),0,0)
    print(mask)
    y[(mask < -1.0) ] = np.nan
    return y
x = np.arange(-3.0, 5.0, 0.1)
plt.figure(figsize=(2,5))
plt.plot(x, func2(x), 'k')
plt.show

In [ ]:
df = pd.read_csv('imputed_relERT_costs_50d.csv').drop(['repetition'],axis=1)
#df['min']= df.min(axis=1)
df = pd.melt(df, id_vars=['dim','fid'], value_vars=df.columns.drop(['dim','fid']))

sns.set_theme(style="ticks")
f, ax = plt.subplots(figsize=(7, 6))
ax.set_yscale("log")
sns.boxplot(data=df[(df.fid<25)&(df.dim==10)],
            y='value',
            x='fid',
            width=.6,
            whis=[0, 100]
           )
ax.xaxis.grid(True)
ax.set(ylabel="")
sns.despine(trim=True, left=True)

Architecture evaluation

In [ ]:
import torch.nn as nn
import torch.nn.functional as F
import torch
from gymnasium import spaces

from stable_baselines3.common.torch_layers import BaseFeaturesExtractor

m = nn.Conv1d(1, 16, 4)
n = nn.Conv1d(1, 16, 4, dilation=3)
infeatures_fix = ['ic.h_max', 'ic.eps_s', 'ic.eps_max', 'ic.eps_ratio', 'ic.m0',
       'ela_meta.lin_simple.adj_r2', 'ela_meta.lin_simple.intercept',
       'ela_meta.lin_simple.coef.min', 'ela_meta.lin_simple.coef.max',
       'ela_meta.lin_simple.coef.max_by_min',
       'ela_meta.lin_w_interact.adj_r2', 'ela_meta.quad_simple.adj_r2',
       'ela_meta.quad_simple.cond', 'ela_meta.quad_w_interact.adj_r2',
       'ela_distr.skewness', 'ela_distr.kurtosis',
       'ela_distr.number_of_peaks', 'nbc.nn_nb.sd_ratio',
       'nbc.nn_nb.mean_ratio', 'nbc.nn_nb.cor',
       'nbc.dist_ratio.coeff_var', 'nbc.nb_fitness.cor',
       'disp.ratio_mean_25', 'disp.ratio_median_25', 'disp.diff_mean_25',
       'disp.diff_median_25']
data = torch.tensor(features_test.loc[:,infeatures_fix].values, dtype=torch.float).unsqueeze(1)
output = n(data)
output.shape
class SimpelConvNet(BaseFeaturesExtractor):
    """
    :param observation_space: (gym.Space)
    :param features_dim: (int) Number of features extracted.
        This corresponds to the number of unit for the last layer.
    """
    
    def __init__(self, observation_space: space.Box, features_dim: int = 26):
        super().__init__(observation_space, features_dim)
        #NxCxL batchXchannelXfeatures
        n_channels = observation_space.shape[1]
        self.cnn = nn.Sequential(
            nn.Conv1d(n_channels, 16, 4, kernel_size=3),
            nn.ReLU(),
            nn.Conv1d(n_channels, 16, 4, dilation=3, kernel_size=3),
            nn.ReLU(),
            nn.Flatten(),
        )
        with torch.no_grad():
            n_flatten = self.cnn(
                torch.as_tensor(observation_space.sample()[None]).float()
            ).shape[1]
        self.linear = nn.Sequential(nn.Linear(n_flatten, features_dim), nn.ReLU())
        
    def forward(self, observations: torch.Tensor) -> torch.Tensor:
        return self.linear(self.cnn(observations))

In [ ]:
features_test

In [ ]:
features_test.loc[features_test.dim==3,infeatures_fix]['disp.diff_median_25'].plot(kind='kde')

\[-1,1\]:
- ela_meta.lin_simple.adj_r2
- ela_meta.lin_w_interact.adj_r2
- ela_meta.quad_simple.adj_r2
- ela_meta.quad_w_interact.adj_r2
- ela_distr.number_of_peaks
- disp.ratio_mean_25
- disp.ratio_median_25
- disp.diff_mean_25
- disp.diff_median_25

additionally clip: 75%
- ic.eps_max
- ela_meta.lin_simple.intercept
- ela_meta.lin_simple.coef.min
- ela_meta.lin_simple.coef.max
- ela_meta.lin_simple.coef.max_by_min
- ela_meta.quad_simple.cond

norm:
- ela_distr.skewness
- ela_distr.kurtosis
- nbc.nn_nb.sd_ratio
- nbc.nn_nb.mean_ratio
- nbc.nn_nb.cor
- nbc.dist_ratio.coeff_var
- nbc.nb_fitness.cor